In [1]:
%%javascript
IPython.notebook.clear_all_output();

<IPython.core.display.Javascript object>

In [2]:
%reset -f
from IPython import get_ipython
get_ipython().magic('reset -sf')

%who

Interactive namespace is empty.


In [3]:
import os
import glob
import h5py
import shutil
import imgaug as aug
import numpy as np 
# import pandas as pd 
# import seaborn as sns
# import matplotlib.pyplot as plt
# import matplotlib.image as mimg
import imgaug.augmenters as iaa
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
from pathlib import Path
from skimage.io import imread, imsave
# from skimage.transform import resize
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from mlxtend.plotting import plot_confusion_matrix
# from sklearn.metrics import confusion_matrix
import cv2
# from keras import backend as K
# color = sns.color_palette()
# %matplotlib inline

# import tensorflow as tf
# from tensorflow.keras.models import Sequential, Model
# from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
# from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array, array_to_img
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, SeparableConv2D
# from tensorflow.keras.layers import GlobalMaxPooling2D
# from tensorflow.keras.layers import BatchNormalization
# from tensorflow.keras.layers import Concatenate
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
# from tensorflow.keras.utils import to_categorical
import sys
import datetime
import time

print("Python version: ", sys.version)
print("Version info.: ", sys.version_info)
# print("TensorFlow version: ", tf.__version__)
# print("TensorFlow.Keras version : ", tf.keras.__version__)

Python version:  3.6.10 |Anaconda, Inc.| (default, Mar 23 2020, 23:13:11) 
[GCC 7.3.0]
Version info.:  sys.version_info(major=3, minor=6, micro=10, releaselevel='final', serial=0)


In [4]:
# Set the seed for hash based operations in python
os.environ['PYTHONHASHSEED'] = '0'

# Set the numpy seed
np.random.seed(111)

# Make the augmentation sequence deterministic
aug.seed(111)

In [5]:
# Define path to the data directory
data_dir = Path('/data/user/tr27p/Courses/CS765-DeepLearning/FinalProject/Chest_X-Ray_Images_Pneumonia/Data/chest_xray/chest_xray')

# Path to train directory (Fancy pathlib...no more os.path!!)
train_dir = data_dir / 'train'

# Path to validation directory
val_dir = data_dir / 'val'

# Path to test directory
test_dir = data_dir / 'test'

In [6]:
# Define path to the NEW data directory
new_data_dir = data_dir.parents[1] / 'new_chest_xray'

# Path to train directory (Fancy pathlib...no more os.path!!)
new_train_dir = new_data_dir / 'train'

# Path to validation directory
new_val_dir = new_data_dir / 'val'

# Path to test directory
new_test_dir = new_data_dir / 'test'

In [7]:
# Get the path to the normal sub-directories
train_normal_cases_dir = train_dir / 'NORMAL'
val_normal_cases_dir = val_dir / 'NORMAL'
test_normal_cases_dir = test_dir / 'NORMAL'
# Get the path to the pneumonia sub-directories
train_pneumonia_cases_dir = train_dir / 'PNEUMONIA'
val_pneumonia_cases_dir = val_dir / 'PNEUMONIA'
test_pneumonia_cases_dir = test_dir / 'PNEUMONIA'


# Get the NEW path to the normal sub-directories
new_train_normal_cases_dir = new_train_dir / 'NORMAL'
new_val_normal_cases_dir = new_val_dir / 'NORMAL'
new_test_normal_cases_dir = new_test_dir / 'NORMAL'
# Get the NEW path to the pneumonia sub-directories
new_train_pneumonia_cases_dir = new_train_dir / 'PNEUMONIA'
new_val_pneumonia_cases_dir = new_val_dir / 'PNEUMONIA'
new_test_pneumonia_cases_dir = new_test_dir / 'PNEUMONIA'

In [8]:
brk_count = -1

In [9]:
# Get the list of all the TRAIN images
train_normal_cases = train_normal_cases_dir.glob('*.jpeg')
train_pneumonia_cases = train_pneumonia_cases_dir.glob('*.jpeg')

bool_rt_aug = True
count = 1
for img_name in sorted(train_normal_cases):
    ## Copy Original Img ##
    shutil.copy2(img_name, str(new_train_normal_cases_dir/img_name.name))
    ## Read Img ##
    img = imread(img_name)
    
    ## Shift -1 Column ##
    img_s_1C = np.roll(img, -1, axis=1)
    cv2.imwrite(str(str(new_train_normal_cases_dir/img_name.stem)+"_shifted-1C"+img_name.suffix), img_s_1C)
    ## Shift 2 Column ##
    img_s_2C = np.roll(img, 2, axis=1)
    cv2.imwrite(str(str(new_train_normal_cases_dir/img_name.stem)+"_shifted-2C"+img_name.suffix), img_s_2C)
    
    ## Horizontal flip ##
    flp_aug = iaa.Fliplr()
    img_flp_aug = flp_aug.augment_image(img)
    img_s_1C_flp_aug = flp_aug.augment_image(img_s_1C)
    img_s_2C_flp_aug = flp_aug.augment_image(img_s_2C)
    cv2.imwrite(str(str(new_train_normal_cases_dir/img_name.stem)+"_flip-hr"+img_name.suffix), img_flp_aug)
    cv2.imwrite(str(str(new_train_normal_cases_dir/img_name.stem)+"_shifted-1C-flip-hr"+img_name.suffix), 
                img_s_1C_flp_aug)
    cv2.imwrite(str(str(new_train_normal_cases_dir/img_name.stem)+"_shifted-2C-flip-hr"+img_name.suffix), 
                img_s_2C_flp_aug)
    
    ## Roatation ##
    if bool_rt_aug:
        rt_aug = iaa.Affine(rotate=20)
        bool_rt_aug = False
    else:
        rt_aug = iaa.Affine(rotate=-20)
        bool_rt_aug = True
    img_rt_aug = rt_aug.augment_image(img)
    img_s_2C_flp_aug_rt_aug = rt_aug.augment_image(img_s_2C_flp_aug)
    cv2.imwrite(str(str(new_train_normal_cases_dir/img_name.stem)+"_roatate-20"+img_name.suffix), img_rt_aug)
    cv2.imwrite(str(str(new_train_normal_cases_dir/img_name.stem)+"_shft-2C-flp-hr-roatate-20"+img_name.suffix), 
                img_s_2C_flp_aug_rt_aug)
    
    ## Random Brightness ##
    br_aug = iaa.Multiply((1.2, 1.5))
    img_br_aug = br_aug.augment_image(img)
    img_s_1C_flp_aug_br_aug = br_aug.augment_image(img_s_1C_flp_aug)
    img_s_2C_flp_aug_rt_aug_br_aug = br_aug.augment_image(img_s_2C_flp_aug_rt_aug)
    cv2.imwrite(str(str(new_train_normal_cases_dir/img_name.stem)+"_rnd-bright"+img_name.suffix), img_br_aug)
    cv2.imwrite(str(str(new_train_normal_cases_dir/img_name.stem)+"_shft-1C-flp-hr-rnd-bright"+img_name.suffix),
                img_s_1C_flp_aug_br_aug)
    cv2.imwrite(str(str(new_train_normal_cases_dir/img_name.stem)+"_shft-2C-flp-rtr-rnd-brigt"+img_name.suffix),
                img_s_2C_flp_aug_rt_aug_br_aug)
    
    if count == brk_count:
        break
        
    count +=1

bool_rt_aug = True
count = 1
for img_name in sorted(train_pneumonia_cases):
    ## Copy Original Img ##
    shutil.copy2(img_name, str(new_train_pneumonia_cases_dir/img_name.name))
    ## Read Img ##
    img = imread(img_name)
    
    ## Shift -1 Column ##
    img_s_1C = np.roll(img, -1, axis=1)
    cv2.imwrite(str(str(new_train_pneumonia_cases_dir/img_name.stem)+"_shifted-1C"+img_name.suffix), img_s_1C)
#     ## Shift 2 Column ##
#     img_s_2C = np.roll(img, 2, axis=1)
#     cv2.imwrite(str(str(new_train_pneumonia_cases_dir/img_name.stem)+"_shifted-2C"+img_name.suffix), img_s_2C)
    
    ## Horizontal flip ##
    flp_aug = iaa.Fliplr()
    img_flp_aug = flp_aug.augment_image(img)
    cv2.imwrite(str(str(new_train_pneumonia_cases_dir/img_name.stem)+"_flip-hr"+img_name.suffix), img_flp_aug)
    
    ## Roatation ##
    if bool_rt_aug:
        rt_aug = iaa.Affine(rotate=20)
        bool_rt_aug = False
    else:
        rt_aug = iaa.Affine(rotate=-20)
        bool_rt_aug = True
    img_rt_aug = rt_aug.augment_image(img)
    cv2.imwrite(str(str(new_train_pneumonia_cases_dir/img_name.stem)+"_roatate-20"+img_name.suffix), img_rt_aug)
    
    ## Random Brightness ##
    br_aug = iaa.Multiply((1.2, 1.5))
    img_br_aug = br_aug.augment_image(img)
    cv2.imwrite(str(str(new_train_pneumonia_cases_dir/img_name.stem)+"_rnd-brightness"+img_name.suffix), 
                img_br_aug)
    
    if count == brk_count:
        break
        
    count +=1

In [10]:
# Get the list of all the VALIDATION images
val_normal_cases = val_normal_cases_dir.glob('*.jpeg')
val_pneumonia_cases = val_pneumonia_cases_dir.glob('*.jpeg')

bool_rt_aug = True
count = 1
for img_name in sorted(val_normal_cases):
    ## Copy Original Img ##
    shutil.copy2(img_name, str(new_test_normal_cases_dir/img_name.name))
    ## Read Img ##
    img = imread(img_name)
    
    ## Shift -1 Column ##
    img_s_1C = np.roll(img, -1, axis=1)
    cv2.imwrite(str(str(new_test_normal_cases_dir/img_name.stem)+"_shifted-1C"+img_name.suffix), img_s_1C)
    ## Shift 2 Column ##
    img_s_2C = np.roll(img, 2, axis=1)
    cv2.imwrite(str(str(new_test_normal_cases_dir/img_name.stem)+"_shifted-2C"+img_name.suffix), img_s_2C)
    
    ## Horizontal flip ##
    flp_aug = iaa.Fliplr()
    img_flp_aug = flp_aug.augment_image(img)
    img_s_1C_flp_aug = flp_aug.augment_image(img_s_1C)
    img_s_2C_flp_aug = flp_aug.augment_image(img_s_2C)
    cv2.imwrite(str(str(new_test_normal_cases_dir/img_name.stem)+"_flip-hr"+img_name.suffix), img_flp_aug)
    cv2.imwrite(str(str(new_test_normal_cases_dir/img_name.stem)+"_shifted-1C-flip-hr"+img_name.suffix), 
                img_s_1C_flp_aug)
    cv2.imwrite(str(str(new_test_normal_cases_dir/img_name.stem)+"_shifted-2C-flip-hr"+img_name.suffix), 
                img_s_2C_flp_aug)
    
    ## Roatation ##
    if bool_rt_aug:
        rt_aug = iaa.Affine(rotate=20)
        bool_rt_aug = False
    else:
        rt_aug = iaa.Affine(rotate=-20)
        bool_rt_aug = True
    img_rt_aug = rt_aug.augment_image(img)
    img_s_2C_flp_aug_rt_aug = rt_aug.augment_image(img_s_2C_flp_aug)
    cv2.imwrite(str(str(new_test_normal_cases_dir/img_name.stem)+"_roatate-20"+img_name.suffix), img_rt_aug)
    cv2.imwrite(str(str(new_test_normal_cases_dir/img_name.stem)+"_shft-2C-flp-hr-roatate-20"+img_name.suffix), 
                img_s_2C_flp_aug_rt_aug)
    
    ## Random Brightness ##
    br_aug = iaa.Multiply((1.2, 1.5))
    img_br_aug = br_aug.augment_image(img)
    img_s_1C_flp_aug_br_aug = br_aug.augment_image(img_s_1C_flp_aug)
    img_s_2C_flp_aug_rt_aug_br_aug = br_aug.augment_image(img_s_2C_flp_aug_rt_aug)
    cv2.imwrite(str(str(new_test_normal_cases_dir/img_name.stem)+"_rnd-bright"+img_name.suffix), img_br_aug)
    cv2.imwrite(str(str(new_test_normal_cases_dir/img_name.stem)+"_shft-1C-flp-hr-rnd-bright"+img_name.suffix),
                img_s_1C_flp_aug_br_aug)
    cv2.imwrite(str(str(new_test_normal_cases_dir/img_name.stem)+"_shft-2C-flp-rtr-rnd-brigt"+img_name.suffix),
                img_s_2C_flp_aug_rt_aug_br_aug)
    
    if count == brk_count:
        break
        
    count +=1

bool_rt_aug = True
count = 1
for img_name in sorted(val_pneumonia_cases):
    ## Copy Original Img ##
    shutil.copy2(img_name, str(new_test_pneumonia_cases_dir/img_name.name))
    ## Read Img ##
    img = imread(img_name)
    
    ## Shift -1 Column ##
    img_s_1C = np.roll(img, -1, axis=1)
    cv2.imwrite(str(str(new_test_pneumonia_cases_dir/img_name.stem)+"_shifted-1C"+img_name.suffix), img_s_1C)
    ## Shift 2 Column ##
    img_s_2C = np.roll(img, 2, axis=1)
    cv2.imwrite(str(str(new_test_pneumonia_cases_dir/img_name.stem)+"_shifted-2C"+img_name.suffix), img_s_2C)
    
    ## Horizontal flip ##
    flp_aug = iaa.Fliplr()
    img_flp_aug = flp_aug.augment_image(img)
    cv2.imwrite(str(str(new_test_pneumonia_cases_dir/img_name.stem)+"_flip-hr"+img_name.suffix), img_flp_aug)
    
    ## Roatation ##
    if bool_rt_aug:
        rt_aug = iaa.Affine(rotate=20)
        bool_rt_aug = False
    else:
        rt_aug = iaa.Affine(rotate=-20)
        bool_rt_aug = True
    img_rt_aug = rt_aug.augment_image(img)
    cv2.imwrite(str(str(new_test_pneumonia_cases_dir/img_name.stem)+"_roatate-20"+img_name.suffix), img_rt_aug)
    
    ## Random Brightness ##
    br_aug = iaa.Multiply((1.2, 1.5))
    img_br_aug = br_aug.augment_image(img)
    cv2.imwrite(str(str(new_test_pneumonia_cases_dir/img_name.stem)+"_rnd-brightness"+img_name.suffix),
                img_br_aug)
    
    if count == brk_count:
        break
        
    count +=1

In [11]:
# Get the list of all the TEST images
test_normal_cases = test_normal_cases_dir.glob('*.jpeg')
test_pneumonia_cases = test_pneumonia_cases_dir.glob('*.jpeg')

bool_rt_aug = True
count = 1
for img_name in sorted(test_normal_cases):
    ## Copy Original Img ##
    shutil.copy2(img_name, str(new_val_normal_cases_dir/img_name.name))
    ## Read Img ##
    img = imread(img_name)
    
    ## Shift -1 Column ##
    img_s_1C = np.roll(img, -1, axis=1)
    cv2.imwrite(str(str(new_val_normal_cases_dir/img_name.stem)+"_shifted-1C"+img_name.suffix), img_s_1C)
    ## Shift 2 Column ##
    img_s_2C = np.roll(img, 2, axis=1)
    cv2.imwrite(str(str(new_val_normal_cases_dir/img_name.stem)+"_shifted-2C"+img_name.suffix), img_s_2C)
    
    ## Horizontal flip ##
    flp_aug = iaa.Fliplr()
    img_flp_aug = flp_aug.augment_image(img)
    img_s_1C_flp_aug = flp_aug.augment_image(img_s_1C)
    img_s_2C_flp_aug = flp_aug.augment_image(img_s_2C)
    cv2.imwrite(str(str(new_val_normal_cases_dir/img_name.stem)+"_flip-hr"+img_name.suffix), img_flp_aug)
    cv2.imwrite(str(str(new_val_normal_cases_dir/img_name.stem)+"_shifted-1C-flip-hr"+img_name.suffix), 
                img_s_1C_flp_aug)
    cv2.imwrite(str(str(new_val_normal_cases_dir/img_name.stem)+"_shifted-2C-flip-hr"+img_name.suffix), 
                img_s_2C_flp_aug)
    
    ## Roatation ##
    if bool_rt_aug:
        rt_aug = iaa.Affine(rotate=20)
        bool_rt_aug = False
    else:
        rt_aug = iaa.Affine(rotate=-20)
        bool_rt_aug = True
    img_rt_aug = rt_aug.augment_image(img)
    img_s_2C_flp_aug_rt_aug = rt_aug.augment_image(img_s_2C_flp_aug)
    cv2.imwrite(str(str(new_val_normal_cases_dir/img_name.stem)+"_roatate-20"+img_name.suffix), img_rt_aug)
    cv2.imwrite(str(str(new_val_normal_cases_dir/img_name.stem)+"_shft-2C-flp-hr-roatate-20"+img_name.suffix), 
                img_s_2C_flp_aug_rt_aug)
    
    ## Random Brightness ##
    br_aug = iaa.Multiply((1.2, 1.5))
    img_br_aug = br_aug.augment_image(img)
    img_s_1C_flp_aug_br_aug = br_aug.augment_image(img_s_1C_flp_aug)
    img_s_2C_flp_aug_rt_aug_br_aug = br_aug.augment_image(img_s_2C_flp_aug_rt_aug)
    cv2.imwrite(str(str(new_val_normal_cases_dir/img_name.stem)+"_rnd-bright"+img_name.suffix), img_br_aug)
    cv2.imwrite(str(str(new_val_normal_cases_dir/img_name.stem)+"_shft-1C-flp-hr-rnd-bright"+img_name.suffix),
                img_s_1C_flp_aug_br_aug)
    cv2.imwrite(str(str(new_val_normal_cases_dir/img_name.stem)+"_shft-2C-flp-rtr-rnd-brigt"+img_name.suffix),
                img_s_2C_flp_aug_rt_aug_br_aug)
    
    if count == brk_count:
        break
        
    count +=1

bool_rt_aug = True
count = 1
for img_name in sorted(test_pneumonia_cases):
    ## Copy Original Img ##
    shutil.copy2(img_name, str(new_val_pneumonia_cases_dir/img_name.name))
    ## Read Img ##
    img = imread(img_name)
    
    ## Shift -1 Column ##
    img_s_1C = np.roll(img, -1, axis=1)
    cv2.imwrite(str(str(new_val_pneumonia_cases_dir/img_name.stem)+"_shifted-1C"+img_name.suffix), img_s_1C)
    ## Shift 2 Column ##
    img_s_2C = np.roll(img, 2, axis=1)
    cv2.imwrite(str(str(new_val_pneumonia_cases_dir/img_name.stem)+"_shifted-2C"+img_name.suffix), img_s_2C)
    
    ## Horizontal flip ##
    flp_aug = iaa.Fliplr()
    img_flp_aug = flp_aug.augment_image(img)
    cv2.imwrite(str(str(new_val_pneumonia_cases_dir/img_name.stem)+"_flip-hr"+img_name.suffix), img_flp_aug)
    
    ## Roatation ##
    if bool_rt_aug:
        rt_aug = iaa.Affine(rotate=20)
        bool_rt_aug = False
    else:
        rt_aug = iaa.Affine(rotate=-20)
        bool_rt_aug = True
    img_rt_aug = rt_aug.augment_image(img)
    cv2.imwrite(str(str(new_val_pneumonia_cases_dir/img_name.stem)+"_roatate-20"+img_name.suffix), img_rt_aug)
    
    ## Random Brightness ##
    br_aug = iaa.Multiply((1.2, 1.5))
    img_br_aug = br_aug.augment_image(img)
    cv2.imwrite(str(str(new_val_pneumonia_cases_dir/img_name.stem)+"_rnd-brightness"+img_name.suffix), 
                img_br_aug)
    
    if count == brk_count:
        break
        
    count +=1